In [0]:
import os
import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd/content/drive/My Drive/tobigs_chatbot/

/content/drive/My Drive/tobigs_chatbot


In [4]:
df = pd.read_pickle("./khaiii_data.pkl")
df.head()

,Question,Answer,Q_token,A_token,len_Q_token,len_A_token
0,휴대폰 판매점이 지적장애인을 속여 휴대폰 개통대전휴대폰 판매점 직원 지적장애 1급 ...,형법 제348조준사기 제1항 미성년자의 지려천박 또는 사람의 심신장애를 이용하여 재...,"[휴대폰/NNG, 판매점/NNG + 이/JKS, 지적/NNG + 장애인/NNG + ...","[형법/NNG, 제/XPN + 348/SN + 조/NNB + 준사기/NNG, 제/X...",28,26
1,장애등록이 된 학생의 특수교육대상자 의무 지정 여부장애등록이 된 학생은 특수교육대상...,특수교육대상자는 장애인 등에 대한 특수교육법 제15조에 의거 특수교육대상자 선정 유...,"[장애/NNG + 등록/NNG + 이/JKC, 되/VV + ㄴ/ETM, 학생/NNG...","[특수/NNG + 교육/NNG + 대상자/NNG + 는/JX, 장애인/NNG, 등/...",14,31
2,특수교육대상자 배치 기준특수교육대상자 배치 기준은,장애인 등에 대한 특수교육법 제17조 제2항에 따라 교육장 또는 교육감은 특수교육대...,"[특수/NNG + 교육/NNG + 대상자/NNG, 배치/NNG, 기준/NNG + 특...","[장애인/NNG, 등/NNB + 에/JKB, 대하/VV + ㄴ/ETM, 특수/NNG...",5,22
3,특수교육대상자 선정에 필요한 서류신규로 특수교육대상자로 선정되기를 희망합니다 이 때...,진단평가 의뢰서 배치신청서 개인정보 수집이용 동의서 기타 참고자료기초조사서 주소지 ...,"[특수/NNG + 교육/NNG + 대상자/NNG, 선정/NNG + 에/JKB, 필요...","[진단/NNG + 평가/NNG, 의뢰서/NNG, 배치/NNG + 신청서/NNG, 개...",12,22
4,발달장애인 주간활동서비스 신청 대상은 어떻게 되나요발달장애인 주간활동지원서비스 신청...,만18세64세의 장애인복지법상 등록된 지적 자폐성 장애인이 대상입니다성인 발달장애인...,"[발달장애/NNG + 이/VCP + ㄴ/ETM, 주간/NNG + 활동/NNG + 서...",[만/NNG + 18/SN + 세/NNB + 64/SN + 세/NNB + 의/JKG...,11,24


In [5]:
print('전체 학습데이터의 개수: {}'.format(len(df)))

전체 학습데이터의 개수: 9232


In [0]:
def token_split(x):
    return [z for i in [i.split("+") for i in x] for z in i]

In [0]:
df['Q_token'] = df['Q_token'].apply(token_split)
df['A_token'] = df['A_token'].apply(token_split)

In [0]:
df['len_Q_token'] = df['Q_token'].apply(lambda x : len(x))
df['len_A_token'] = df['A_token'].apply(lambda x : len(x))

In [9]:
df.describe()

,len_Q_token,len_A_token
count,9232.000000,9232.000000
mean,40.273614,156.299285
std,59.520797,504.213402
min,1.000000,1.000000
25%,17.000000,39.000000
50%,26.000000,78.000000
75%,42.000000,165.000000
max,1537.000000,34174.000000


In [0]:
# 0.25 ~0.8분위수 크기의 token만 뽑음.
thre_under = 0.25
thre_over = 0.8

Q_thre_under = np.quantile(df['len_Q_token'], thre_under)
Q_thre_over = np.quantile(df['len_Q_token'], thre_over)

A_thre_under = np.quantile(df['len_A_token'], thre_under)
A_thre_over = np.quantile(df['len_A_token'], thre_over)

In [0]:
df = df[(df['len_A_token'] <= A_thre_over) & (df['len_Q_token'] <= Q_thre_over) & 
                  (df['len_A_token'] >= 10 ) & (df['len_Q_token'] >= 10 )].reset_index(drop = True)

In [12]:
question = df['Q_token'].tolist()
answer = df['A_token'].tolist()

Q_corpus = [z for i in df['Q_token'].tolist() for z in i]
A_corpus = [z for i in df['A_token'].tolist() for z in i]
total_corpus = Q_corpus+A_corpus

len(Q_corpus), len(A_corpus), len(total_corpus)

(129377, 378268, 507645)

In [13]:
df.head()

,Question,Answer,Q_token,A_token,len_Q_token,len_A_token
0,장애등록이 된 학생의 특수교육대상자 의무 지정 여부장애등록이 된 학생은 특수교육대상...,특수교육대상자는 장애인 등에 대한 특수교육법 제15조에 의거 특수교육대상자 선정 유...,"[장애/NNG , 등록/NNG , 이/JKC, 되/VV , ㄴ/ETM, 학생/...","[특수/NNG , 교육/NNG , 대상자/NNG , 는/JX, 장애인/NNG,...",34,73
1,특수교육대상자 배치 기준특수교육대상자 배치 기준은,장애인 등에 대한 특수교육법 제17조 제2항에 따라 교육장 또는 교육감은 특수교육대...,"[특수/NNG , 교육/NNG , 대상자/NNG, 배치/NNG, 기준/NNG ,...","[장애인/NNG, 등/NNB , 에/JKB, 대하/VV , ㄴ/ETM, 특수/N...",11,49
2,특수교육대상자 선정에 필요한 서류신규로 특수교육대상자로 선정되기를 희망합니다 이 때...,진단평가 의뢰서 배치신청서 개인정보 수집이용 동의서 기타 참고자료기초조사서 주소지 ...,"[특수/NNG , 교육/NNG , 대상자/NNG, 선정/NNG , 에/JKB,...","[진단/NNG , 평가/NNG, 의뢰서/NNG, 배치/NNG , 신청서/NNG,...",32,42
3,발달장애인 주간활동서비스 신청 대상은 어떻게 되나요발달장애인 주간활동지원서비스 신청...,만18세64세의 장애인복지법상 등록된 지적 자폐성 장애인이 대상입니다성인 발달장애인...,"[발달장애/NNG , 이/VCP , ㄴ/ETM, 주간/NNG , 활동/NNG ...","[만/NNG , 18/SN , 세/NNB , 64/SN , 세/NNB , ...",26,59
4,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요장애인활동...,장애인활동지원 홈페이지wwwableserviceorkr에서 지정된 활동지원기관 및 ...,"[장애인활동/NNG , 지원/NNG, 교육/NNG , 기관/NNG , 과/JC...","[장애인활동/NNG , 지원/NNG, 홈페이지/NNG , wwwableservi...",40,23


In [0]:
import re 

In [0]:
df['Q_token'] = [[j.replace(' ','').split('/')[0] for j in x] for x in df['Q_token']]

In [16]:
title_questions = df['Q_token'].tolist()
title_questions

[['장애',
  '등록',
  '이',
  '되',
  'ㄴ',
  '학생',
  '의',
  '특수',
  '교육',
  '대상자',
  '의무',
  '지정',
  '여부',
  '장애',
  '등록',
  '이',
  '되',
  'ㄴ',
  '학생',
  '은',
  '특수',
  '교육',
  '대상자',
  '로',
  '의무',
  '적',
  '으로',
  '지정',
  '이',
  '되',
  '는',
  '것',
  '이',
  'ㄴ가요'],
 ['특수', '교육', '대상자', '배치', '기준', '특수', '교육', '대상자', '배치', '기준', '은'],
 ['특수',
  '교육',
  '대상자',
  '선정',
  '에',
  '필요',
  '하',
  'ㄴ',
  '서류',
  '신규',
  '로',
  '특수',
  '교육',
  '대상자',
  '로',
  '선정',
  '되',
  '기',
  '를',
  '희망',
  '하',
  'ㅂ니다',
  '이',
  '때',
  '필요',
  '하',
  'ㄴ',
  '서류',
  '는',
  '무엇',
  '이',
  'ㄴ가요'],
 ['발달장애',
  '이',
  'ㄴ',
  '주간',
  '활동',
  '서비스',
  '신청',
  '대상',
  '은',
  '어떻',
  '게',
  '되',
  '나요',
  '발달장애',
  '이',
  'ㄴ',
  '주간',
  '활동',
  '지원서비스',
  '신청',
  '대상',
  '은',
  '어떻',
  '게',
  '되',
  '나요'],
 ['장애인활동',
  '지원',
  '교육',
  '기관',
  '과',
  '제공',
  '기관',
  '에',
  '대하',
  'ㄴ',
  '정보',
  '는',
  '어디',
  '에서',
  '확인',
  '하',
  'ㄹ',
  '수',
  '있',
  '나요',
  '장애인활동',
  '지원',
  '교육',
  '기관',
  '과',
  '제공',
  '기관',
  '

In [0]:
df['A_token'] = [[j.replace(' ','').split('/')[0] for j in x] for x in df['A_token']]

In [18]:
title_answers = df['A_token'].tolist()
title_answers

[['특수',
  '교육',
  '대상자',
  '는',
  '장애인',
  '등',
  '에',
  '대하',
  'ㄴ',
  '특수',
  '교육법',
  '제',
  '15',
  '조',
  '에',
  '의거',
  '특수',
  '교육',
  '대상자',
  '선정',
  '유형',
  '의',
  '어느',
  '하나',
  '에',
  '해당',
  '하',
  '는',
  '사람',
  '중',
  '특수',
  '교육',
  '을',
  '필요',
  '로',
  '하',
  '는',
  '사람',
  '으로',
  '진단',
  '평가',
  '되',
  'ㄴ',
  '사람',
  '을',
  '말',
  '하',
  'ㅂ니다',
  '따라서',
  '특수',
  '교육',
  '대상자',
  '로',
  '선정',
  '되',
  '기',
  '를',
  '희망',
  '하',
  '는',
  '학생',
  '은',
  '특수',
  '교육',
  '대상자',
  '진단',
  '평가',
  '를',
  '신청',
  '하',
  '여야',
  '되',
  'ㅂ니다'],
 ['장애인',
  '등',
  '에',
  '대하',
  'ㄴ',
  '특수',
  '교육법',
  '제',
  '17',
  '조',
  '제',
  '2',
  '항',
  '에',
  '따르',
  '아',
  '교육장',
  '또는',
  '교육감',
  '은',
  '특수',
  '교육',
  '대상자',
  '의',
  '장애',
  '정도능력',
  '보호자',
  '의',
  '의견',
  '등',
  '을',
  '종합',
  '적',
  '으로',
  '판단',
  '하',
  '여',
  '거주지',
  '에서',
  '가장',
  '가깝',
  'ㄴ',
  '곳',
  '에',
  '배치',
  '하',
  '여야',
  '하',
  'ㅂ니다'],
 ['진단',
  '평가',
  '의뢰서',
  '배치',
  '신청서',
  '개인',
  '정보',

# Word2Vec (Skip-Gram)

In [19]:
from gensim.models import Word2Vec
model_data = title_questions
model = Word2Vec(size=2, window=4, min_count=5, workers=-1, sg=1, iter=100)
# build vocab
model.build_vocab(model_data)
# 새로운 데이터가  들어오면  trian으로
# model.train(new_data, total_examples=model.corpus_count, epochs=100) 

# model.wv = word vector object
# 단어 dictionary
vocabs = list(model.wv.vocab.keys())
vectors = [model.wv[i] for i in vocabs]
X = [i[0] for i in vectors]
Y = [i[1] for i in vectors]
# word vector dataframe 
w2v_df = pd.DataFrame({'X':X, 'Y':Y}, index=vocabs)
w2v_df.head()

,X,Y
장애,-0.146805,-0.036599
등록,0.129602,-0.118621
이,0.193665,0.082485
되,-0.163922,-0.035527
ㄴ,-0.014801,-0.107667


In [20]:
w2v_df.shape

(1768, 2)

# Fasttext

In [21]:
from gensim.models import FastText
model_data = title_questions
model = FastText(size=2, window=4, min_count=5, workers=-1, sg=1, iter=100)
# build vocab
model.build_vocab(model_data)
# 새로운 데이터가  들어오면  trian으로
# model.train(new_data, total_examples=model.corpus_count, epochs=100) 

# model.wv = word vector object
# 단어 dictionary
vocabs = list(model.wv.vocab.keys())
vectors = [model.wv[i] for i in vocabs]
X = [i[0] for i in vectors]
Y = [i[1] for i in vectors]
# word vector dataframe 
ft_df = pd.DataFrame({'X':X, 'Y':Y}, index=vocabs)
ft_df.head()

,X,Y
장애,-0.146805,-0.036599
등록,0.129602,-0.118621
이,0.193665,0.082485
되,-0.163922,-0.035527
ㄴ,-0.014801,-0.107667


# Embedding Ensemble

In [22]:
print(w2v_df.shape, ft_df.shape)
concat_df = pd.concat((w2v_df, ft_df)) # aixs=0으로
new_df = concat_df.groupby(concat_df.index) # 인덱스로 묶어서 mean처리해줌
df_means = new_df.mean()
df_means

(1768, 2) (1768, 2)


,X,Y
1,-0.065078,-0.022891
10,0.048402,0.091000
100,0.242039,-0.057641
11,-0.236853,-0.032376
12,0.117721,0.049509
...,...,...
휴일,-0.160742,-0.100701
희,-0.148913,0.001605
희망,0.064250,-0.158406
힘,-0.202975,-0.045881


# Document Embedding

In [23]:
words = df_means.index.values.tolist()
lookup = df_means.values
lookup_dict = dict(zip(words, lookup))
embed_size = df_means.shape[1]
lookup_dict

{'1': array([-0.06507821, -0.02289071]),
 '10': array([0.04840206, 0.09100019]),
 '100': array([ 0.24203928, -0.05764054]),
 '11': array([-0.23685305, -0.03237611]),
 '12': array([0.1177213, 0.0495088]),
 '13': array([ 0.02378507, -0.01083335]),
 '14': array([-0.08250966, -0.03995222]),
 '15': array([0.0666727 , 0.07223252]),
 '18': array([0.17563652, 0.1282047 ]),
 '19': array([0.21990545, 0.06552553]),
 '2': array([0.2482831 , 0.06331832]),
 '20': array([-0.09271096, -0.07697061]),
 '200': array([-0.16953929,  0.24901019]),
 '2000': array([0.24201016, 0.14914234]),
 '2011': array([ 0.23902927, -0.04153043]),
 '2014': array([ 0.03414297, -0.06302211]),
 '2015': array([-0.16187094,  0.05771287]),
 '2016': array([-0.18573582, -0.20448776]),
 '2017': array([0.01503845, 0.2284397 ]),
 '2018': array([-0.13050391,  0.01994779]),
 '2019': array([-0.00622142, -0.02328749]),
 '2020': array([ 0.18993551, -0.2087661 ]),
 '2022': array([-0.06569578, -0.14697769]),
 '22': array([-0.10731521, -0.17

In [0]:
words = df_means.index.values.tolist()
lookup = df_means.values.tolist()
lookup_dict = dict(zip(words, lookup))
embed_size = df_means.shape[1]

# token데이터와 임베딩 사전 교집합 
new = []
for i in model_data:
    inner = list(set(i).intersection(set(words)))
    new.append(inner)
    

sentence_embbeding = np.zeros((embed_size,))
for sentence in new:
    embedding = np.zeros((embed_size,))
    for word in sentence:
        # 각 벡터들 평균
        embedding += lookup_dict[word]
        embedding = embedding / len(sentence)
    sentence_embbeding = np.vstack([sentence_embbeding, embedding])
sentence_embbeding = sentence_embbeding[1:,:]

In [25]:
# N 번째 질문의 정보
def getMeta(num):
    print(f"Token : {model_data[num]}\n")
    #print(f"Tf-Idf : {df.iloc[num][df.iloc[num] > 0]}\n")
    print(f"Sentence Embed : {sentence_embbeding[num]}\n")
    
    dist_li = [getDist(sentence_embbeding[num], sentence_embbeding[i]) \
               for i in range(sentence_embbeding.shape[0]) if i != num]
    nearest_vector = min(dist_li)
    nearest_num =  dist_li.index(min(dist_li))
    print(f'가장 가까운 문서 토큰 : {model_data[nearest_num]}')
    print(f'가장 가까운 문서 거리 : {nearest_vector}')
            
def getDist(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))   

getMeta(1)

Token : ['특수', '교육', '대상자', '배치', '기준', '특수', '교육', '대상자', '배치', '기준', '은']

Sentence Embed : [-0.01583418  0.02703395]

가장 가까운 문서 토큰 : ['지적', '장애', '3', '급', '도', '대출', '받', '을', '수', '있', '나', '요지', '적', '장애', '3', '급', '도', '대출', '받', '을', '수', '있', '나요']
가장 가까운 문서 거리 : 0.00028685908801906137


# Doc2vec

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
df["index"] = [str(i) for i in np.arange(len(df))]

In [27]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d,c) for d, c in df[['Q_token', 'index']].values]
print(len(tagged_train_docs))

5153


In [28]:
tagged_train_docs

[TaggedDocument(words=['장애', '등록', '이', '되', 'ㄴ', '학생', '의', '특수', '교육', '대상자', '의무', '지정', '여부', '장애', '등록', '이', '되', 'ㄴ', '학생', '은', '특수', '교육', '대상자', '로', '의무', '적', '으로', '지정', '이', '되', '는', '것', '이', 'ㄴ가요'], tags='0'),
 TaggedDocument(words=['특수', '교육', '대상자', '배치', '기준', '특수', '교육', '대상자', '배치', '기준', '은'], tags='1'),
 TaggedDocument(words=['특수', '교육', '대상자', '선정', '에', '필요', '하', 'ㄴ', '서류', '신규', '로', '특수', '교육', '대상자', '로', '선정', '되', '기', '를', '희망', '하', 'ㅂ니다', '이', '때', '필요', '하', 'ㄴ', '서류', '는', '무엇', '이', 'ㄴ가요'], tags='2'),
 TaggedDocument(words=['발달장애', '이', 'ㄴ', '주간', '활동', '서비스', '신청', '대상', '은', '어떻', '게', '되', '나요', '발달장애', '이', 'ㄴ', '주간', '활동', '지원서비스', '신청', '대상', '은', '어떻', '게', '되', '나요'], tags='3'),
 TaggedDocument(words=['장애인활동', '지원', '교육', '기관', '과', '제공', '기관', '에', '대하', 'ㄴ', '정보', '는', '어디', '에서', '확인', '하', 'ㄹ', '수', '있', '나요', '장애인활동', '지원', '교육', '기관', '과', '제공', '기관', '에', '대하', 'ㄴ', '정보', '는', '어디', '에서', '확인', '하', 'ㄹ', '수', '있', '나요'], tags='4'),
 

In [0]:
doc_vectorizer = Doc2Vec(
    dm=0,           
    dbow_words=1,    
    window=4,       
    size=100,       
    alpha=0.025,     
    seed=0,
    min_count=3,    
    min_alpha=0.7, 
    workers=-1)   

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

doc_vectorizer.build_vocab(tagged_train_docs)
X = np.array([doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs])
y = np.array([int(doc.tags) for doc in tagged_train_docs])

In [31]:
X

array([[ 0.0016144 , -0.00424824, -0.00107246, ...,  0.00273921,
        -0.00040103,  0.00281918],
       [ 0.00073993,  0.00117504,  0.00493087, ...,  0.0002765 ,
         0.00494269,  0.00222982],
       [-0.0034    ,  0.00278055,  0.00392918, ...,  0.00055715,
         0.00111106, -0.00328785],
       ...,
       [ 0.00452848, -0.00224779,  0.00493103, ...,  0.00017835,
         0.00427037, -0.00332268],
       [-0.00157691, -0.00017158,  0.00274976, ...,  0.00031117,
         0.00214173, -0.00062728],
       [-0.00376134, -0.00069469,  0.00421649, ...,  0.00311127,
         0.00087575, -0.00484626]], dtype=float32)

In [32]:
def cosSim(num):
    print('Question :', df.iloc[num]['Question'])
    A_sparse = sparse.csr_matrix(X)
    similarities = cosine_similarity(A_sparse)
    nearest_num = np.argsort(similarities[num,:])[::-1][1]
    print('Answer :', df.iloc[nearest_num]['Answer'])
cosSim(0)

Question : 장애등록이 된 학생의 특수교육대상자 의무 지정 여부장애등록이 된 학생은 특수교육대상자로 의무적으로 지정이 되는 것인가요
Answer :  1 장애인용 차량 취득세 및 자동차세 감면 장애등급 13급시각장애인의 경우 4급의 장애인이 보철용 또는 생업활동용으로 사용하기 위하여 취득하는 차량으로서 아래의 에 해당되는 차량 중 최초로 감면 신청하는 1대에 한하여 면제  배기량 2000cc이하의 승용자동차  승차정원 7인승이상 10인승이하인 승용자동차  승차정원 15인승 이하의 승합자동차  적재적량 1톤이하 화물자동차 이륜자동차  한 세대에 13급인 장애인이 2인 이상이면서 각각 자동차를 취득소유하고 있을 경우 장애인마다 차량 1대씩 지방세를 면제받을 수 있음 


In [0]:
from gensim.test.utils import get_tmpfile

# fname = get_tmpfile('./my_doc2vec_model')

doc_vectorizer.save('./my_doc2vec_model')

In [0]:
model = Doc2Vec.load('./my_doc2vec_model')

# New Question (Input)

In [0]:
Question = input()
Question

장애인 연금을 받기 위해서는 무조건 재심사를 거쳐야 하는건가요?


'장애인 연금을 받기 위해서는 무조건 재심사를 거쳐야 하는건가요?'

In [0]:
!git clone https://github.com/kakao/khaiii.git

In [0]:
!pip install cmake

In [0]:
!mkdir build

In [0]:
!cd build && cmake /content/khaiii

In [0]:
!cd /content/build/ && make all

In [0]:
!cd /content/build/ && make resource

In [0]:
!cd /content/build && make install

In [0]:
!cd /content/build && make package_python

In [0]:
!pip install /content/build/package_python

In [0]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

new_question = []
for word in api.analyze(Question):
    for morph in word.morphs:
        new_question.append(morph.lex)

In [0]:
new_question

['장애인',
 '연금',
 '을',
 '받',
 '기',
 '위하',
 '여서',
 '는',
 '무조건',
 '재심사',
 '를',
 '거치',
 '어야',
 '하',
 '는',
 '것',
 '이',
 'ㄴ가요',
 '?']

In [0]:
# token데이터와 임베딩 사전 교집합 
new = []
inner = list(set(new_question).intersection(set(words)))
new.append(inner)

for sentence in new:
    embedding = np.zeros((embed_size,))
    for word in sentence:
        # 각 벡터들 평균
        embedding += lookup_dict[word]
        embedding = embedding / len(sentence)
embedding

array([-0.00751466,  0.01373715])

In [0]:
# N 번째 질문의 정보
def test_evaluate(ques):
    print("Token :" ,ques)
    print("Sentence Embed :", embedding)
    
    dist_li = [getDist(embedding, sentence_embbeding[i]) \
               for i in range(sentence_embbeding.shape[0]) if i != new_question]
    nearest_vector = min(dist_li)
    nearest_num =  dist_li.index(min(dist_li))
    print(f'가장 가까운 문서 토큰 : {model_data[nearest_num]}')
    print(f'가장 가까운 문서 거리 : {nearest_vector}\n')

    print('Question :', Question)
    print('Answer :', df.iloc[nearest_num]['Answer'])
            
def getDist(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))   

test_evaluate(new_question)

Token : ['장애인', '연금', '을', '받', '기', '위하', '여서', '는', '무조건', '재심사', '를', '거치', '어야', '하', '는', '것', '이', 'ㄴ가요', '?']
Sentence Embed : [-0.00751466  0.01373715]
가장 가까운 문서 토큰 : ['장애인복지', '카드', '재', '발급', '받', '을려고', '하', '는데', '재', '발급', '받', '을', '때', '돈', '받', '나요', '장애', '인복지', '카드', '재', '발급', '받', '을려고', '하', '는데', '재', '발급', '받', '을', '때', '돈', '받나', '요']
가장 가까운 문서 거리 : 0.0002310723161726242

Question : 장애인 연금을 받기 위해서는 무조건 재심사를 거쳐야 하는건가요?
Answer :  기존의 장애인등록증복지카드 포함에 장애인 고속도로 통행료 할인기능을 통합한 통합복지카드 A형은 발급비용 4000원으로 기존 고속도로통행료 카드 신청비용과 동일하며 수납은 주소지 관할 읍면동 주민센터에서 이루어 집니다 신용카드와 직불카드 기능이 있는 통합복지카드 B형 발급비용은 별도로 없으며 신한카드사에서 제작 비용을 부담합니다 


# QA_machine

In [0]:
def QnA(question_input):
  Question = input('질문을 작성하시오')

  !git clone https://github.com/kakao/khaiii.git
  !pip install cmake
  !mkdir build
  !cd build && cmake /content/khaiii
  !cd /content/build/ && make all
  !cd /content/build/ && make resource
  !cd /content/build && make install
  !cd /content/build && make package_python
  !pip install /content/build/package_python

  from khaiii import KhaiiiApi
  api = KhaiiiApi()

  new_question = []
  for word in api.analyze(Question):
    for morph in word.morphs:
        new_question.append(morph.lex)

  # token데이터와 임베딩 사전 교집합 
  new = []
  inner = list(set(new_question).intersection(set(words)))
  new.append(inner)

  for sentence in new:
      embedding = np.zeros((embed_size,))
      for word in sentence:
          # 각 벡터들 평균
          embedding += lookup_dict[word]
          embedding = embedding / len(sentence)

  # N 번째 질문의 정보
  def test_evaluate(ques):
      print("Token :" ,ques)
      print("Sentence Embed :", embedding)
      
      dist_li = [getDist(embedding, sentence_embbeding[i]) \
                for i in range(sentence_embbeding.shape[0]) if i != new_question]
      nearest_vector = min(dist_li)
      nearest_num =  dist_li.index(min(dist_li))
      print(f'가장 가까운 문서 토큰 : {model_data[nearest_num]}')
      print(f'가장 가까운 문서 거리 : {nearest_vector}\n')

      print('Question :', Question)
      print('Answer :', df.iloc[nearest_num]['Answer'])
              
  def getDist(p1, p2):
      return np.sqrt(np.sum((p1 - p2) ** 2))   

  return test_evaluate(new_question)


In [0]:
QnA( )

질문을 작성하시오장애인주차구역에 일반차량이 장애인탑승시키고 주차시켜도 되나요
Cloning into 'khaiii'...
remote: Enumerating objects: 877, done.
remote: Total 877 (delta 0), reused 0 (delta 0), pack-reused 877
Receiving objects: 100% (877/877), 30.21 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (358/358), done.
Checking out files: 100% (131/131), done.
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test fma_compiles
-- Performing Test fma_compiles - Success
-- Performi

Token : ['장애인주차구역', '에', '일반', '차량', '이', '장애인탑승', '시키', '고', '주차', '시키', '어도', '되', '나요']
Sentence Embed : [-0.017508   -0.01873755]
가장 가까운 문서 토큰 : ['지적', '장애', '가', '후천', '적', '으로', '되', 'ㄹ', '수', '있', '나요', '지', '적', '장애', '가', '후천', '적', '으로', '되', 'ㄹ', '수', '있', '나요']
가장 가까운 문서 거리 : 0.00019253993023672693

Question : 장애인주차구역에 일반차량이 장애인탑승시키고 주차시켜도 되나요
Answer :  지적장애는 선천적이기도 하지만 후천적인 경우도 있습니다 태어날 때는 아무 이상이 없고 건강했다가도 어떤 외부적 요인으로 인해 지적장애가 되기도 합니다 
